## For clarity and smoother interpretation of the results, only the model cells that achieved the highest accuracies have been retained. Other hyperparameter trials were removed.

# Installing Required Libraries & Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [ ]:
# Installing needed libraries
!pip install -q transformers datasets

# Importing needed libraries
import shutil
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

from datasets import Dataset, DatasetDict, load_dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)

from torch.utils.data import DataLoader, TensorDataset


# Preprocessing Sourashtra Dataset

In [ ]:
# Load dataset
#file_path = "/content/Sourashtra Dataset.xlsx"  # Uploaded dataset on my drive instead of shared drives
file_path = "/content/drive/MyDrive/Sourashtra Dataset.xlsx"
sheet_names = ["Positive", "Negative"]

# Preprocessing
# Read both sheets and assign column names manually
df_pos = pd.read_excel(file_path, sheet_name=sheet_names[0], header=None)
df_neg = pd.read_excel(file_path, sheet_name=sheet_names[1], header=None)
df_pos.columns = ["Text", "Label"]
df_neg.columns = ["Text", "Label"]

# Merge datasets
sourashtra_df = pd.concat([df_pos, df_neg], ignore_index=True)

# Checking to see loaded dataframe
print(sourashtra_df.head(20))

# Shuffle the dataset to mix positive and negative samples
sourashtra_df = sourashtra_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the shuffled dataset
print(sourashtra_df.head(20))


                                          Text  Label
0                     thora chokkay chokkat se      0
1                       Mogo gullayo ven oppay      0
2                     Thora ton singaar ken se      0
3                   Elle kaathayo mogo opaaras      0
4                         Tu onde chokkat meni      0
5                    Tu kobeem mogo help kerey      0
6   Askitheno chokkat monu sethe meni me hovdu      0
7                      Tu gavaryo mogo opaaras      0
8                          Aski chokkat chaley      0
9                          Hindo chokkat dinnu      0
10                 Tu kobeem moro lovtho raano      0
11            Elle sawlo togo chokkat dekaaras      0
12          Tu ave athudu mogo sondos avudraas      0
13                  Thora holo praamo kerarthe      0
14                 Mogo bharaat kalathe oppaey      0
15                       Me thora haath kaalus      0
16                    Thora gavlo nenav ken se      0
17             Thora khes ch

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

for i in range(5):  # Check first 5 examples
    text = sourashtra_df["Text"].iloc[i]
    tokens = tokenizer.tokenize(text)
    print(f"\nOriginal Text: {text}")
    print(f"Tokens: {tokens}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]


Original Text: Senian mathri se
Tokens: ['▁Seni', 'an', '▁ma', 'thri', '▁se']

Original Text: Askitheno chokkat monu sethe meni me hovdu
Tokens: ['▁Ask', 'ithe', 'no', '▁cho', 'kka', 't', '▁mon', 'u', '▁se', 'the', '▁meni', '▁me', '▁', 'hov', 'du']

Original Text: Mora sangaadi kobeem ra
Tokens: ['▁Mora', '▁sang', 'a', 'adi', '▁ko', 'be', 'em', '▁ra']

Original Text: Togo kai pajje? Me devus
Tokens: ['▁To', 'go', '▁kai', '▁paj', 'je', '?', '▁Me', '▁de', 'vus']

Original Text: Enno kobeem athudu meni bathi hovde
Tokens: ['▁En', 'no', '▁ko', 'be', 'em', '▁ath', 'udu', '▁meni', '▁bath', 'i', '▁', 'hov', 'de']


# Baseline Accuracy

In [ ]:
# Tokenization
# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [ ]:
# Evaluation

print(sourashtra_df["Label"].value_counts())

model.eval()
preds, true_labels = [], []
with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1) #CHANGE HERE: added zero_diviison

# Display results
print(f"Baseline Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}") #values: precision = 0.500 meaning model predicts wrong half the time, the recall is 1.00, meaning model captures all true positives but not true negatives or misclassifies these


# Combine results into DataFrame
results_df = pd.DataFrame({
    "Text": sourashtra_df["Text"].iloc[:len(true_labels)].values,
    "Ground Truth": true_labels,
    "Predicted": preds
})

# Show top rows
print("\nSample Predictions:")
print(results_df)

# Optional: Save to CSV
results_df.to_csv("/content/drive/My Drive/sourashtra_predictions_baselines.csv", index=False)

Label
1    125
0    125
Name: count, dtype: int64
Baseline Accuracy: 0.5000
Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667

Sample Predictions:
                                           Text  Ground Truth  Predicted
0                              Senian mathri se             1          1
1    Askitheno chokkat monu sethe meni me hovdu             0          1
2                       Mora sangaadi kobeem ra             0          1
3                      Togo kai pajje? Me devus             0          1
4           Enno kobeem athudu meni bathi hovde             0          1
..                                          ...           ...        ...
245                 Tu elle kerithe mogo oppaes             0          1
246                 Mogo bharaat kalathe oppaey             0          1
247              Elle nanna suno singaar ken se             0          1
248           Thone dhoo, pisi madiri dekhaaras             1          1
249               Enno kobeem askiyo hedithothe 

# Preprocessing Indo-Aryan Datasets

In [ ]:
# first step is to import all relevant datasets into dataframes and makes sure their output columns are same format

#clean marathi dataset
# import pandas as pd

marathi_df = pd.read_csv('/content/marathi.csv') # 1 = positive, 0 = neutral, -1 = negative

marathi_df_mod = marathi_df.loc[marathi_df['label'] != 0].copy()

marathi_df_mod.loc[:, 'label'] = marathi_df_mod['label'].replace(-1, 0) # to have 0 and 1 labels

marathi_df_mod = marathi_df_mod.drop(marathi_df_mod.columns[0], axis=1) # to remove an unnecessary indexes column

marathi_df_mod = marathi_df_mod.rename(columns={
    "marathi_sentence": "Text",
    "label": "Label"
})

print(marathi_df_mod)

                                                    Text  Label
0      माने यांचा घटस्फोट झाला आहे तर मोहितेने नुकतेच...      0
1      एका रात्रीत घडणारी किंबहुना बिघडणारी ही गोष्ट आहे      0
2      जरी आघात समजण्यायोग्य आहे जरी चित्रपटाला खराब ...      0
3      पण तो असा आघातही अनुभवत आहे की तो कोणाशीही शेअ...      0
4                   छोटे-छोटे गैरसमज मोठ्या अडचणीत येतात      0
...                                                  ...    ...
11995                   बालकलाकारांनी उत्तम काम केले आहे      1
11996                       भूमिकेतील बारकावे पकडले आहेत      1
11997  चिंतन आणि त्याचा बाबा यांचं हे भावस्पर्शी जग य...      1
11998  काही वेळा त्याग दाखवता दाखवता विनोद होऊन जाण्य...      1
11999        दिग्गज नीना कुलकर्णी आणि संजय मोने सहज आहेत      1

[8000 rows x 2 columns]


In [ ]:
#we might need to go in and clean this dataset bc it seems to have emojis and some english text as well. but first try it without modifying
# import pandas as pd

gujarati_df = pd.read_csv('/content/gujarati.csv') # Non-hostile': 0 (positive), 'Hostile': 1 (negative), we need to encode these categorically

gujarati_df['Label'] = gujarati_df['Label'].map({'Non-hostile': 1, 'Hostile': 0}) #now positive sentime = 1, negative sentimet = 0

print(gujarati_df)

                                                    Text  Label
0      સંયુક્ત રાષ્ટ્રસંઘના વિશેષ દૂત રિચર્ડ બેનેટે અ...      1
1      કોઈ પણ સમાજને આગળ વધવા માટે જે પણ ક્ષેત્રે મદદ...      1
2      આતંકવાદી યાસિન મલિકને આજીવન કેદની સજા : ટેરર ફ...      1
3      ▶️જમ્મુ કાશ્મીર - ભારતીય સેનાએ કુપવાડામાં 3 આત...      1
4      @Tarun20tha મોદી પણ દેશ ચલાવે છે તો કેમ આતંકવા...      1
...                                                  ...    ...
14677  Bjp ના પોલીસ ના નખરા નઈ સહન કરવાના વિરોધ ની જર...      0
14678  BJP ના લૂંટારા નેતા ઓ ની સાચેજ હવે ફાટવા લાગી ...      0
14679  Bjp ના લોકો પાસે દંડ માંગવાની તેવડ નથી ગુલામી ...      0
14680      Bjp ને વોટ ના આપો હજી ખરાબ હાલત કરશે આ બધા \n      0
14681  BJP સરકાર માં પોલીસ ને પોતાની માં બહેનો ની પણ ...      0

[14682 rows x 2 columns]


In [ ]:
print(gujarati_df['Label'].value_counts())

Label
1    7400
0    7282
Name: count, dtype: int64


In [ ]:
## hindi dataset, quite small, but worthy to test

from datasets import load_dataset

ds = load_dataset("OdiaGenAI/sentiment_analysis_hindi")

hindi_df = pd.DataFrame(ds['train'])  # Assuming you're using the 'train' split

# Map labels: 'neg' → -1, 'neu' → 0, 'pos' → 1
hindi_df['label'] = hindi_df['label'].map({'neg': -1, 'neu': 0, 'pos': 1})

# Filter out neutral labels (keep only -1 and 1)
hindi_df = hindi_df[hindi_df['label'] != 0]

hindi_df['label'] = hindi_df['label'].replace(-1, 0)

hindi_df = hindi_df.rename(columns={
    "text": "Text",
    "label": "Label"
})

print(hindi_df)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

sentiment_analysis_term_train.jsonl:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2497 [00:00<?, ? examples/s]

      Label                                               Text
0         0  हमें इससे बेहतर ब्राइटनेस वाले टेबलेट देखने को...
1         1                          बैटरी लाइफ बहुत बढिया है।
3         1  स्लोफो एक मनोरंजक एप्लिकेशन है इसमें कोई दो रा...
4         1                             अच्छी बैटरी क्षमता है।
5         1  बैटरी लाइफ की बात करें तो हमारे लगातार वीडियो ...
...     ...                                                ...
2488      1  गूगल नाउ खास आपके लोकेशन के आधार पर चीजों को स...
2489      0                     धीमी गति से ऑटो-फोकस होता  है।
2490      1  टैबलेट पर लगी स्पीकर ग्रिल इसकी खूबसूरती में औ...
2492      0  हां, इसमें ड्यूल सिम फंक्शनैलिटी न होना, कइयों...
2495      0  हालांकि, हमने पाया कि इसका टच कभी-कभी भटक जाता...

[1498 rows x 2 columns]


# Finetuning with Indo-Aryan Languages (Linguistically Similar)



## FineTune with Gujarati

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split

# # Load dataset
# gujarati_df = pd.read_csv('/content/gujarati.csv')

# # Encode labels: 'Non-hostile' -> 1 (Positive), 'Hostile' -> 0 (Negative)
# gujarati_df['Label'] = gujarati_df['Label'].map({'Non-hostile': 1, 'Hostile': 0})

# Shuffle dataset
gujarati_df = gujarati_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    gujarati_df["Text"], gujarati_df["Label"], test_size=0.2, random_state=42
)

# Convert to strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format to PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and learning rate scheduler
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    save_total_limit=1,  # Only keep best model
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",  # 🚨 Disables wandb/logging tools
)

# Compute evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train model (Results will print in Colab)
trainer.train()

# Evaluate final model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)

In [ ]:
# Save the fine-tuned model
trainer.save_model("./xlm-roberta-gujarati-best")

# Move model to Google Drive
shutil.move("./xlm-roberta-gujarati-best", "/content/xlm-roberta-gujarati-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/xlm-roberta-gujarati-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Gujarati:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")


## Finetuning with Marathi

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split

# Shuffle dataset
marathi_df_mod = marathi_df_mod.sample(frac=1, random_state=42).reset_index(drop=True)

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    marathi_df_mod["Text"], marathi_df_mod["Label"], test_size=0.2, random_state=42
)

# Convert to strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format to PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and learning rate scheduler
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    save_total_limit=1,  # Only keep best model
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",  # 🚨 Disables wandb/logging tools
)

# Compute evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train model (Results will print in Colab)
trainer.train()

# Evaluate final model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-2833c97868ea>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Final Evaluation Results: {'eval_loss': 0.3189987242221832, 'eval_accuracy': 0.873125, 'eval_precision': 0.8901960784313725, 'eval_recall': 0.85125, 'eval_f1': 0.8702875399361022, 'eval_runtime': 20.8724, 'eval_samples_per_second': 76.656, 'eval_steps_per_second': 4.791, 'epoch': 3.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./xlm-roberta-marathi-best")

# Move model to Google Drive
shutil.move("./xlm-roberta-marathi-best", "/content/xlm-roberta-marathi-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/xlm-roberta-marathi-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Marathi:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Marathi:
➡️ Accuracy: 0.5680
➡️ Precision: 0.5630, Recall: 0.6080, F1-score: 0.5846


## Finetuning with Hindi

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split

# Shuffle dataset
hindi_df = hindi_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    hindi_df["Text"], hindi_df["Label"], test_size=0.2, random_state=42
)

# Convert to strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format to PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#  Define optimizer and learning rate scheduler
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    save_total_limit=1,  # Only keep best model
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",  # 🚨 Disables wandb/logging tools
)

# Compute evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train model (Results will print in Colab)
trainer.train()

# Evaluate final model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)

Map:   0%|          | 0/1198 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-5fb939f51258>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Final Evaluation Results: {'eval_loss': 0.28772833943367004, 'eval_accuracy': 0.93, 'eval_precision': 0.9605263157894737, 'eval_recall': 0.948051948051948, 'eval_f1': 0.954248366013072, 'eval_runtime': 3.8363, 'eval_samples_per_second': 78.2, 'eval_steps_per_second': 4.953, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./xlm-roberta-hindi-best")

# Move model to Google Drive
shutil.move("./xlm-roberta-hindi-best", "/content/xlm-roberta-hindi-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/xlm-roberta-hindi-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi:
➡️ Accuracy: 0.5240
➡️ Precision: 0.5158, Recall: 0.7840, F1-score: 0.6222


### Ensemble Method with All models evaluated on Sourashtra

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Define model paths
model_paths = {
    "marathi": "/content/xlm-roberta-marathi-best",
    "hindi": "/content/xlm-roberta-hindi-best",
    "gujarati": "/content/xlm-roberta-gujarati-best"
}

# Load models
models = {
    name: AutoModelForSequenceClassification.from_pretrained(path).to(device).eval()
    for name, path in model_paths.items()
}

# Load Sourashtra dataset
# sourashtra_df = pd.read_csv("/content/sourashtra_data.csv")
texts = sourashtra_df["Text"].astype(str).tolist()
true_labels = sourashtra_df["Label"].tolist()

# Tokenize
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

# Collect logits
with torch.no_grad():
    all_logits = []
    for model in models.values():
        outputs = model(**encodings)
        all_logits.append(outputs.logits)

# Soft voting: average logits
mean_logits = torch.mean(torch.stack(all_logits), dim=0)
preds = torch.argmax(mean_logits, dim=1).cpu().numpy()

# Print evaluation
accuracy = accuracy_score(true_labels, preds)
print(f"\nAccuracy on Sourashtra data: {accuracy * 100:.2f}%\n")
print(classification_report(true_labels, preds, digits=4))

# Combine results into DataFrame
results_df = pd.DataFrame({
    "Text": texts,
    "Ground Truth": true_labels,
    "Predicted": preds
})

# Show top rows
print("\nSample Predictions:")
print(results_df)

# Optional: Save to CSV
# results_df.to_csv("/content/sourashtra_predictions_ensemble.csv", index=False)
results_df.to_csv("/content/drive/My Drive/5541_Results/sourashtra_predictions_ensemble.csv", index=False)


Accuracy on Sourashtra data: 53.20%

              precision    recall  f1-score   support

           0     0.6053    0.1840    0.2822       125
           1     0.5189    0.8800    0.6528       125

    accuracy                         0.5320       250
   macro avg     0.5621    0.5320    0.4675       250
weighted avg     0.5621    0.5320    0.4675       250


Sample Predictions:
                                           Text  Ground Truth  Predicted
0                              Senian mathri se             1          1
1    Askitheno chokkat monu sethe meni me hovdu             0          1
2                       Mora sangaadi kobeem ra             0          1
3                      Togo kai pajje? Me devus             0          0
4           Enno kobeem athudu meni bathi hovde             0          1
..                                          ...           ...        ...
245                 Tu elle kerithe mogo oppaes             0          0
246                 Mogo bhara

## seeing marathi predictions

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_path = "/content/xlm-roberta-marathi-best"
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Load Sourashtra dataset
# Make sure sourashtra_df is already loaded
texts = sourashtra_df["Text"].astype(str).tolist()
true_labels = sourashtra_df["Label"].tolist()

# Tokenize
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

# Predict
with torch.no_grad():
    outputs = model(**encodings)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).cpu().numpy()

# Evaluation
accuracy = accuracy_score(true_labels, preds)
print(f"\n✅ Marathi model on Sourashtra data — Accuracy: {accuracy * 100:.2f}%")
print(classification_report(true_labels, preds, digits=4))

# Save results to CSV
results_df = pd.DataFrame({
    "Text": texts,
    "Ground Truth": true_labels,
    "Predicted (Marathi Model)": preds
})

# Save path in your Drive
save_path = "/content/drive/My Drive/5541_Results/sourashtra_marathi_only.csv"
results_df.to_csv(save_path, index=False)


✅ Marathi model on Sourashtra data — Accuracy: 56.80%
              precision    recall  f1-score   support

           0     0.5739    0.5280    0.5500       125
           1     0.5630    0.6080    0.5846       125

    accuracy                         0.5680       250
   macro avg     0.5684    0.5680    0.5673       250
weighted avg     0.5684    0.5680    0.5673       250



## Trying out Augmented Finetuning on Gujarati (50 samples)

In [ ]:
# Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=25, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=25, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split


# Combine Gujarati + few-shot Sourashtra for training
combined_train_df = pd.concat([gujarati_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

#Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train!
trainer.train()

# Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Gujarati:", eval_results)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-33-20a5c6cac60c>:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Gujarati: {'eval_loss': 0.8874073624610901, 'eval_accuracy': 0.695, 'eval_precision': 0.6423357664233577, 'eval_recall': 0.88, 'eval_f1': 0.7426160337552743, 'eval_runtime': 2.5441, 'eval_samples_per_second': 78.613, 'eval_steps_per_second': 5.11, 'epoch': 5.0}


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-gujarati-best")

# Move model to Google Drive
# shutil.move("./fewshot-gujarati-best", "/content/fewshot-gujarati-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-gujarati-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Gujarati FEWSHOT:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Gujarati FEWSHOT:
➡️ Accuracy: 0.6950
➡️ Precision: 0.6423, Recall: 0.8800, F1-score: 0.7426


## Augmented Finetuning Marathi

In [ ]:
# Combine Marath + few-shot Sourashtra for training
combined_train_df = pd.concat([marathi_df_mod, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train!
trainer.train()

# Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Marathi:", eval_results)

Map:   0%|          | 0/8050 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-36-0dbec99b8a5c>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Marathi: {'eval_loss': 1.2957309484481812, 'eval_accuracy': 0.685, 'eval_precision': 0.6796116504854369, 'eval_recall': 0.7, 'eval_f1': 0.6896551724137931, 'eval_runtime': 2.5459, 'eval_samples_per_second': 78.559, 'eval_steps_per_second': 5.106, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-marathi-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-marathi-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Marathi FEWSHOT:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Marathi FEWSHOT:
➡️ Accuracy: 0.6850
➡️ Precision: 0.6796, Recall: 0.7000, F1-score: 0.6897


## Augmented Finetuning Hindi

In [ ]:
# Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train!
trainer.train()

# Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1548 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-38-238cb44dde26>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.3108503818511963, 'eval_accuracy': 0.615, 'eval_precision': 0.5751633986928104, 'eval_recall': 0.88, 'eval_f1': 0.6956521739130435, 'eval_runtime': 2.5226, 'eval_samples_per_second': 79.283, 'eval_steps_per_second': 5.153, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT:
➡️ Accuracy: 0.6150
➡️ Precision: 0.5752, Recall: 0.8800, F1-score: 0.6957


### Experiment: Augmented Finetuning Hindi (with different sample sizes)

This experiment was done to find out whether sample sizes of Sourashtra affect the performance of the Augmented Hindi model. Further analysis is included in the report.

Sample Size = 2

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=1, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=1, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 2.1377313137054443, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 105.7423, 'eval_samples_per_second': 2.345, 'eval_steps_per_second': 0.151, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best2")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best2"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 2:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 2:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


Sample Size = 4

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=2, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=2, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1502 [00:00<?, ? examples/s]

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.162582278251648, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 107.3289, 'eval_samples_per_second': 2.292, 'eval_steps_per_second': 0.149, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best4")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best4"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 4:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 4:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


Sample Size = 6

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=3, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=3, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1504 [00:00<?, ? examples/s]

Map:   0%|          | 0/244 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-15-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 0.9202539324760437, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 108.1053, 'eval_samples_per_second': 2.257, 'eval_steps_per_second': 0.148, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best6")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best6"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 6:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/244 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 6:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


Sample Size = 8

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=4, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=4, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1506 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.9741237163543701, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 0.9917355371900827, 'eval_f1': 0.6648199445983379, 'eval_runtime': 100.9826, 'eval_samples_per_second': 2.396, 'eval_steps_per_second': 0.158, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best8")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best8"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 8:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 8:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 0.9917, F1-score: 0.6648


Sample Size = 10, First Run

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=5, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=5, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.3195124864578247, 'eval_accuracy': 0.6375, 'eval_precision': 0.5891891891891892, 'eval_recall': 0.9083333333333333, 'eval_f1': 0.7147540983606557, 'eval_runtime': 109.9589, 'eval_samples_per_second': 2.183, 'eval_steps_per_second': 0.136, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best10")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best10"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 10:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 10:
➡️ Accuracy: 0.6375
➡️ Precision: 0.5892, Recall: 0.9083, F1-score: 0.7148


Sample Size = 10, Second Run

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df10 = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=5, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=5, random_state=42)
])
sourashtra_test_df10 = sourashtra_df.drop(few_shot_df10.index)

# Then save to a folder inside your Drive
few_shot_df10.to_csv("/content/drive/MyDrive/few_shot_sourashtra10.csv", index=False)
sourashtra_test_df10.to_csv("/content/drive/MyDrive/sourashtra_test10.csv", index=False)


In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df10]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df10["Text"].astype(str)
val_labels = sourashtra_test_df10["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    seed=42 # added for consistency
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-c6a842af7e2c>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 0.8388599157333374, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 96.5169, 'eval_samples_per_second': 2.487, 'eval_steps_per_second': 0.155, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best10")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best10"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df10)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 10:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 10:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


Sample Size = 12

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=6, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=6, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1510 [00:00<?, ? examples/s]

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 0.7148350477218628, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 105.49, 'eval_samples_per_second': 2.256, 'eval_steps_per_second': 0.142, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best12")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best12"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 12:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 12:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


Sample Size = 14

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=7, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=7, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1512 [00:00<?, ? examples/s]

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-22-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 0.8308678269386292, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 117.5124, 'eval_samples_per_second': 2.008, 'eval_steps_per_second': 0.128, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best14")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best14"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 14:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 14:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


Sample Size = 16

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df16 = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=8, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=8, random_state=42)
])
sourashtra_test_df16 = sourashtra_df.drop(few_shot_df16.index)

# Then save to a folder inside your Drive
few_shot_df16.to_csv("/content/drive/MyDrive/few_shot_sourashtra16.csv", index=False)
sourashtra_test_df16.to_csv("/content/drive/MyDrive/sourashtra_test16.csv", index=False)


In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df16]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df16["Text"].astype(str)
val_labels = sourashtra_test_df16["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    seed=42
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1514 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-f3b10ccf9051>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.4861992597579956, 'eval_accuracy': 0.6324786324786325, 'eval_precision': 0.5906432748538012, 'eval_recall': 0.8632478632478633, 'eval_f1': 0.7013888888888888, 'eval_runtime': 108.9924, 'eval_samples_per_second': 2.147, 'eval_steps_per_second': 0.138, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best16")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best16"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df16)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 16:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 16:
➡️ Accuracy: 0.6325
➡️ Precision: 0.5906, Recall: 0.8632, F1-score: 0.7014


Sample Size = 18

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df18 = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=9, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=9, random_state=42)
])
sourashtra_test_df18 = sourashtra_df.drop(few_shot_df18.index)

# Then save to a folder inside your Drive
few_shot_df18.to_csv("/content/drive/MyDrive/few_shot_sourashtra18.csv", index=False)
sourashtra_test_df18.to_csv("/content/drive/MyDrive/sourashtra_test18.csv", index=False)


In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df18]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df18["Text"].astype(str)
val_labels = sourashtra_test_df18["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    seed=42
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-7c516b6a8cdb>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 0.8224760890007019, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 92.6742, 'eval_samples_per_second': 2.503, 'eval_steps_per_second': 0.162, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best18")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best18"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df18)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 18:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 18:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


The first run on sample size = 20 yielded 65.25% accuracy, but was lost.

Sample Size = 20, Second Run

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df20 = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=10, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=10, random_state=42)
])
sourashtra_test_df20 = sourashtra_df.drop(few_shot_df20.index)

# Then save to a folder inside your Drive
few_shot_df20.to_csv("/content/drive/MyDrive/few_shot_sourashtra20.csv", index=False)
sourashtra_test_df20.to_csv("/content/drive/MyDrive/sourashtra_test20.csv", index=False)


In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df20]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df20["Text"].astype(str)
val_labels = sourashtra_test_df20["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-98ea606a9358>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 0.8461251854896545, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 96.2422, 'eval_samples_per_second': 2.39, 'eval_steps_per_second': 0.156, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best20")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best20"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df20)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 20:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 20:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


Sample Size = 20, Second Run

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df20_new = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=10, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=10, random_state=42)
])
sourashtra_test_df20_new = sourashtra_df.drop(few_shot_df20_new.index)

# Then save to a folder inside your Drive
few_shot_df20_new.to_csv("/content/drive/MyDrive/few_shot_sourashtra20.csv", index=False)
sourashtra_test_df20_new.to_csv("/content/drive/MyDrive/sourashtra_test20.csv", index=False)


In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df20_new]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df20["Text"].astype(str)
val_labels = sourashtra_test_df20["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-21-bd6cbb2c4002>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.035231113433838, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 100.9444, 'eval_samples_per_second': 2.278, 'eval_steps_per_second': 0.149, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best20_new")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best20_new"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df20_new)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 20:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 20:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


## Ensemble Method Few Shot Transfer

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Define model paths
model_paths = {
    "marathi": "/content/fewshot-marathi-best",
    "hindi": "/content/fewshot-hindi-best",
    "gujarati": "/content/fewshot-gujarati-best"
}

# Load models
models = {
    name: AutoModelForSequenceClassification.from_pretrained(path).to(device).eval()
    for name, path in model_paths.items()
}

# Load Sourashtra test dataset
texts = sourashtra_test_df["Text"].astype(str).tolist()
true_labels = sourashtra_test_df["Label"].tolist()

# Tokenize
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

# Collect logits
with torch.no_grad():
    all_logits = []
    for model in models.values():
        outputs = model(**encodings)
        all_logits.append(outputs.logits)

# Soft voting: average logits
mean_logits = torch.mean(torch.stack(all_logits), dim=0)
preds = torch.argmax(mean_logits, dim=1).cpu().numpy()

# Print evaluation
accuracy = accuracy_score(true_labels, preds)
print(f"\nAccuracy on Sourashtra Test data: {accuracy * 100:.2f}%\n")
print(classification_report(true_labels, preds, digits=4))

# Combine results into DataFrame
results_df = pd.DataFrame({
    "Text": texts,
    "Ground Truth": true_labels,
    "Predicted": preds
})

# Show top rows
print("\nSample Predictions:")
print(results_df)

# Optional: Save to CSV
# results_df.to_csv("/content/sourashtra_predictions_ensemble.csv", index=False)
results_df.to_csv("/content/drive/My Drive/5541_Results/sourashtra_predictions_fewshot_ensemble.csv", index=False)


Accuracy on Sourashtra Test data: 66.00%

              precision    recall  f1-score   support

           0     0.7759    0.4500    0.5696       100
           1     0.6127    0.8700    0.7190       100

    accuracy                         0.6600       200
   macro avg     0.6943    0.6600    0.6443       200
weighted avg     0.6943    0.6600    0.6443       200


Sample Predictions:
                                                  Text  Ground Truth  \
0                              Mora sangaadi kobeem ra             0   
1                             Togo kai pajje? Me devus             0   
2                  Enno kobeem athudu meni bathi hovde             0   
3               Tu keni sangathe vel mogo raag avaraas             1   
4    Bhaat maviyor theno melli thora somaar chokkat...             1   
..                                                 ...           ...   
195                   Thora sangaadi ratha mo oppaaras             0   
196                        Tu ell

## Augmented Testing (Different Sample Sizes) on Indo-Aryan Langauges

In [ ]:
# # ✅ Load dataset
gujarati_df = pd.read_csv('/content/gujarati.csv')

# # ✅ Encode labels: 'Non-hostile' -> 1 (Positive), 'Hostile' -> 0 (Negative)
gujarati_df['Label'] = gujarati_df['Label'].map({'Non-hostile': 1, 'Hostile': 0})

# ✅ Shuffle dataset
gujarati_df = gujarati_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# ✅ Few-shot sampling from Sourashtra
few_shot_df = pd.concat([
    sourashtra_df[sourashtra_df['Label'] == 1].sample(n=5, random_state=42),
    sourashtra_df[sourashtra_df['Label'] == 0].sample(n=5, random_state=42)
])
sourashtra_test_df = sourashtra_df.drop(few_shot_df.index)

In [ ]:
# ✅ Combine Gujarati + few-shot Sourashtra for training
combined_train_df = pd.concat([gujarati_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Gujarati:", eval_results)

Map:   0%|          | 0/14692 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-f8295ae0ba13>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Gujarati: {'eval_loss': 2.2263479232788086, 'eval_accuracy': 0.6541666666666667, 'eval_precision': 0.6, 'eval_recall': 0.925, 'eval_f1': 0.7278688524590164, 'eval_runtime': 6.2759, 'eval_samples_per_second': 38.242, 'eval_steps_per_second': 2.39, 'epoch': 10.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("/content/drive/MyDrive/fewshot-gujarati-best2")

# Move model to Google Drive
# shutil.move("./fewshot-gujarati-best", "/content/fewshot-gujarati-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/drive/MyDrive/fewshot-gujarati-best2"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Gujarati FEWSHOT 5:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Gujarati FEWSHOT 5:
➡️ Accuracy: 0.6542
➡️ Precision: 0.6000, Recall: 0.9250, F1-score: 0.7279


In [ ]:
# ✅ Combine Marath + few-shot Sourashtra for training
combined_train_df = pd.concat([marathi_df_mod, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Marathi:", eval_results)

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-6be2a1cab3b5>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Marathi: {'eval_loss': 1.7172929048538208, 'eval_accuracy': 0.6333333333333333, 'eval_precision': 0.5879120879120879, 'eval_recall': 0.8916666666666667, 'eval_f1': 0.7086092715231788, 'eval_runtime': 6.2683, 'eval_samples_per_second': 38.288, 'eval_steps_per_second': 2.393, 'epoch': 10.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-marathi-best3")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-marathi-best3"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Marathi FEWSHOT 5:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Marathi FEWSHOT 5:
➡️ Accuracy: 0.6333
➡️ Precision: 0.5879, Recall: 0.8917, F1-score: 0.7086


In [ ]:
#BEST
# ✅ Combine Marath + few-shot Sourashtra for training
combined_train_df = pd.concat([marathi_df_mod, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=12,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Marathi:", eval_results)

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-40-1f992e68264b>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Marathi: {'eval_loss': 2.0811116695404053, 'eval_accuracy': 0.6625, 'eval_precision': 0.6101694915254238, 'eval_recall': 0.9, 'eval_f1': 0.7272727272727273, 'eval_runtime': 6.2552, 'eval_samples_per_second': 38.368, 'eval_steps_per_second': 2.398, 'epoch': 12.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-marathi-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-marathi-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Marathi FEWSHOT 5:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

In [ ]:
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-46-238cb44dde26>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.6176300048828125, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 3.0467, 'eval_samples_per_second': 78.774, 'eval_steps_per_second': 4.923, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 5:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 5:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


In [ ]:
#BEST
# ✅ Combine hindi + few-shot Sourashtra for training
combined_train_df = pd.concat([hindi_df, few_shot_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Prepare text & label columns
train_texts = combined_train_df["Text"].astype(str)
train_labels = combined_train_df["Label"]
val_texts = sourashtra_test_df["Text"].astype(str)
val_labels = sourashtra_test_df["Label"]

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to HuggingFace Datasets
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Prepare for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# ✅ Metrics function
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train!
trainer.train()

# ✅ Final evaluation on Sourashtra test set
eval_results = trainer.evaluate()
print("\nFew shot Evaluation on Sourashtra Test Set Hindi:", eval_results)

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-20-77b3fcad568f>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Few shot Evaluation on Sourashtra Test Set Hindi: {'eval_loss': 1.1138842105865479, 'eval_accuracy': 0.6916666666666667, 'eval_precision': 0.64375, 'eval_recall': 0.8583333333333333, 'eval_f1': 0.7357142857142858, 'eval_runtime': 6.3158, 'eval_samples_per_second': 38.0, 'eval_steps_per_second': 2.375, 'epoch': 7.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./fewshot-hindi-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/fewshot-hindi-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_test_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 5:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Hindi FEWSHOT 5:
➡️ Accuracy: 0.6917
➡️ Precision: 0.6438, Recall: 0.8583, F1-score: 0.7357


### Ensemble Method on Augmented Indo-Aryan Languages

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Define model paths
model_paths = {
    "marathi": "/content/drive/MyDrive/fewshot-marathi-best",
    "hindi": "/content/drive/MyDrive/fewshot-hindi-best",
    "gujarati": "/content/drive/MyDrive/fewshot-gujarati-best"
}

# Load models
models = {
    name: AutoModelForSequenceClassification.from_pretrained(path).to(device).eval()
    for name, path in model_paths.items()
}

# Load Sourashtra test dataset
texts = sourashtra_test_df["Text"].astype(str).tolist()
true_labels = sourashtra_test_df["Label"].tolist()

# Tokenize
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

# Collect logits
with torch.no_grad():
    all_logits = []
    for model in models.values():
        outputs = model(**encodings)
        all_logits.append(outputs.logits)

# Soft voting: average logits
mean_logits = torch.mean(torch.stack(all_logits), dim=0)
preds = torch.argmax(mean_logits, dim=1).cpu().numpy()

# Print evaluation
accuracy = accuracy_score(true_labels, preds)
print(f"\nAccuracy on Sourashtra Test data: {accuracy * 100:.2f}%\n")
print(classification_report(true_labels, preds, digits=4))

# Combine results into DataFrame
results_df = pd.DataFrame({
    "Text": texts,
    "Ground Truth": true_labels,
    "Predicted": preds
})

# Show top rows
print("\nSample Predictions:")
print(results_df)

# Optional: Save to CSV
# results_df.to_csv("/content/sourashtra_predictions_ensemble.csv", index=False)
results_df.to_csv("/content/drive/My Drive/sourashtra_predictions_fewshot_only10_ensemble.csv", index=False)


Accuracy on Sourashtra Test data: 67.08%

              precision    recall  f1-score   support

           0     0.8596    0.4083    0.5537       120
           1     0.6120    0.9333    0.7393       120

    accuracy                         0.6708       240
   macro avg     0.7358    0.6708    0.6465       240
weighted avg     0.7358    0.6708    0.6465       240


Sample Predictions:
                                           Text  Ground Truth  Predicted
0                              Senian mathri se             1          1
1    Askitheno chokkat monu sethe meni me hovdu             0          0
2                       Mora sangaadi kobeem ra             0          1
3                      Togo kai pajje? Me devus             0          1
4           Enno kobeem athudu meni bathi hovde             0          1
..                                          ...           ...        ...
235                 Tu elle kerithe mogo oppaes             0          0
236                 Mogo 

## Testing on Dravidian Languages

In [ ]:
# malayalam dataset
import pandas as pd

malayalam_train = pd.read_csv("malayalam_train.tsv", sep="\t")
malayalam_dev = pd.read_csv("malayalam_dev.tsv", sep="\t")

malayalam_df = pd.concat([malayalam_train, malayalam_dev], ignore_index=True)

malayalam_df["category"] = malayalam_df["category"].astype(str).str.strip().str.lower()

# Keep only 'positive' and 'negative'
malayalam_df = malayalam_df[malayalam_df["category"].isin(["positive", "negative"])].reset_index(drop=True)

malayalam_df["category"] = malayalam_df["category"].map({"negative": 0, "positive": 1})

malayalam_df = malayalam_df.rename(columns={"text": "Text", "category": "Label"})

print(malayalam_df["Label"].value_counts())
malayalam_df



Label
1    2246
0     600
Name: count, dtype: int64


Text  Label
0                hoo mammokka police vesham aaha anthas      1
1        Oru rekshayum illa...kidilam kannu nananjupoyi      1
2                             Ikka     waiting.........      1
3                Raju Ettante Oro Shorttum Ijathi ppwli      1
4      Ettan fansil netti poya aarenkilum undo?    #...      1
...                                                 ...    ...
2841    SAMBAVAM HEVY AKKITTUNDE KATTA WAITING FOR RAJA      1
2842   Mammuka happy birthday stay blessed live long...      1
2843   Mamangam keralam kanda ettavum valliya hit akkum      1
2844   FDFS.. trailer kandu ithu vareyum njanithu pa...      1
2845   1:38 enikkathu inganaaa kelkkunne.. mammoookk...      1

[2846 rows x 2 columns]

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
# telegu dataset
from datasets import load_dataset

ds = load_dataset("ravikumar1478/Sentiment_analysis_on_Telugu_movie_Review_3")

# Convert to pandas DataFrame
telegu_df = ds['train'].to_pandas()

telegu_df = telegu_df[telegu_df["label"].isin([0, 2])].reset_index(drop=True)

telegu_df['label'] = telegu_df['label'].replace(2, 1)

telegu_df = telegu_df.rename(columns={"text": "Text", "label": "Label"})
print(telegu_df["Label"].value_counts())
telegu_df

Label
1    6635
0    4751
Name: count, dtype: int64


Text  Label
0      బాలేక‌పోతే మాత్రం ముందే చెప్పానుగా ఇలా జ‌రుగుత...      0
1      అమిత్ షా ర్యాలీని అడ్డుకోవ‌డానికి జ‌రిగిన ఘ‌ర్...      0
2      హలో గురు ప్రేమ కోసమే అనే పాట ఎంత పాతదైనా తరాలక...      1
3      ఈమ‌ధ్య ఇంట‌ర్వ్యూ కోసం మీడియా ముందుకొచ్చి ఆ ఏం...      0
4                                  అర్ధశతకంతో అలరించాడు.      1
...                                                  ...    ...
11381              డిపార్ట్మెంట్ ని కూడా వార్న్ చేస్తారు      0
11382  ఇటీవల ఈ చిత్ర పోస్టర్‌ విడుదల కాగా, ఇందులో అంజ...      0
11383  ఉదయం దీక్ష కొనసాగిస్తున్న భట్టి విక్రమార్కను అ...      0
11384      మాకు భారతీయ సినిమాల గురించి ఎక్కువగా తెలియదు.      0
11385  టీ20 సిరీస్‌ అనంతరం భారత్‌తో నాలుగు మ్యాచ్‌ల ట...      0

[11386 rows x 2 columns]

In [ ]:
## tamil dataset

from datasets import load_dataset

ds = load_dataset("wlkla/Tamil_first_ready_for_sentiment")

# Convert to pandas DataFrame
tamil_df = ds['train'].to_pandas()

tamil_df["label"] = tamil_df["label"].astype(str).str.strip().str.lower()

# Keep only 'positive' and 'negative'
tamil_df = tamil_df[tamil_df["label"].isin(["positive", "negative"])].reset_index(drop=True)

tamil_df["label"] = tamil_df["label"].map({"negative": 0, "positive": 1})

tamil_df = tamil_df.rename(columns={"query": "Text", "label": "Label"})
print(tamil_df["Label"].value_counts())
tamil_df

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Tamil_first_ready_for_sentiment.csv:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15653 [00:00<?, ? examples/s]

Label
1    10496
0     2027
Name: count, dtype: int64


Label                                               Text
0          0               Enna da ellam avan seyal  Mari iruku
1          0          This movei is just like  ellam avan seyal
2          1  Padam vanthathum 13k dislike pottavaga yellam ...
3          1    Neraya neraya neraya... ... V era level...thala
4          1  wow thavala sema mass....padam oru pundaikum a...
...      ...                                                ...
12518      1      Looking young rajini Anna Fans from Telangana
12519      1         Nee vaa thalaivaa ponggalukku naaga erukko
12520      1  Rajini sir god of Indian cenima thalivaaaaaa s...
12521      1                     Again 1990 rajni sir cute look
12522      1  Pattaya Kilaputhupaa trailer... !!!!! Get Rajn...

[12523 rows x 2 columns]

## finetune on malayalam

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split


# ✅ Shuffle dataset
malayalam_df = malayalam_df.sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    malayalam_df["Text"], malayalam_df["Label"], test_size=0.2, random_state=42
)

# ✅ Convert to strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Remove unnecessary columns and set format to PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Define optimizer and learning rate scheduler
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    save_total_limit=1,  # Only keep best model
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",  # 🚨 Disables wandb/logging tools
)

# ✅ Compute evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train model (Results will print in Colab)
trainer.train()

# ✅ Evaluate final model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/2276 [00:00<?, ? examples/s]

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-9459bb579697>:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Final Evaluation Results: {'eval_loss': 0.3909326493740082, 'eval_accuracy': 0.8596491228070176, 'eval_precision': 0.8884120171673819, 'eval_recall': 0.9366515837104072, 'eval_f1': 0.9118942731277533, 'eval_runtime': 7.4396, 'eval_samples_per_second': 76.617, 'eval_steps_per_second': 4.839, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./xlm-roberta-malayalam-best")

# Move model to Google Drive
shutil.move("./xlm-roberta-malayalam-best", "/content/xlm-roberta-malayalam-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/xlm-roberta-malayalam-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Malayalam:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Malayalam:
➡️ Accuracy: 0.4960
➡️ Precision: 0.4968, Recall: 0.6240, F1-score: 0.5532


## finetune on tamil

In [ ]:
# ✅ Shuffle dataset
tamil_df = tamil_df.sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    tamil_df["Text"], tamil_df["Label"], test_size=0.2, random_state=42
)

# ✅ Convert to strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Remove unnecessary columns and set format to PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Define optimizer and learning rate scheduler
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    save_total_limit=1,  # Only keep best model
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",  # 🚨 Disables wandb/logging tools
)

# ✅ Compute evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train model (Results will print in Colab)
trainer.train()

# ✅ Evaluate final model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)

Map:   0%|          | 0/10018 [00:00<?, ? examples/s]

Map:   0%|          | 0/2505 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-af5e43f3afc0>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Final Evaluation Results: {'eval_loss': 0.423757940530777, 'eval_accuracy': 0.8347305389221557, 'eval_precision': 0.8347305389221557, 'eval_recall': 1.0, 'eval_f1': 0.9099216710182768, 'eval_runtime': 33.3168, 'eval_samples_per_second': 75.187, 'eval_steps_per_second': 4.712, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./xlm-roberta-tamil-best")

# Move model to Google Drive
shutil.move("./xlm-roberta-tamil-best", "/content/xlm-roberta-tamil-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/xlm-roberta-tamil-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Tamil:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Tamil:
➡️ Accuracy: 0.5000
➡️ Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667


## finetune on telegu

In [ ]:
# ✅ Shuffle dataset
telegu_df = telegu_df.sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    telegu_df["Text"], telegu_df["Label"], test_size=0.2, random_state=42
)

# ✅ Convert to strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Remove unnecessary columns and set format to PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Define optimizer and learning rate scheduler
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    save_total_limit=1,  # Only keep best model
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",  # 🚨 Disables wandb/logging tools
)

# ✅ Compute evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train model (Results will print in Colab)
trainer.train()

# ✅ Evaluate final model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)

Map:   0%|          | 0/9108 [00:00<?, ? examples/s]

Map:   0%|          | 0/2278 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-5cefe226b1d2>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Final Evaluation Results: {'eval_loss': 0.31138691306114197, 'eval_accuracy': 0.902107111501317, 'eval_precision': 0.9047619047619048, 'eval_recall': 0.927536231884058, 'eval_f1': 0.9160075329566855, 'eval_runtime': 30.5009, 'eval_samples_per_second': 74.686, 'eval_steps_per_second': 4.688, 'epoch': 5.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("./xlm-roberta-telegu-best")

# Move model to Google Drive
shutil.move("./xlm-roberta-telegu-best", "/content/xlm-roberta-telegu-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/xlm-roberta-telegu-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Telegu:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Telegu:
➡️ Accuracy: 0.5680
➡️ Precision: 0.5612, Recall: 0.6240, F1-score: 0.5909


## ensemble method dravidian

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Define model paths
model_paths = {
    "tamil": "/content/xlm-roberta-tamil-best",
    "telegu": "/content/xlm-roberta-telegu-best",
    "malayalam": "/content/xlm-roberta-malayalam-best"
}

# Load models
models = {
    name: AutoModelForSequenceClassification.from_pretrained(path).to(device).eval()
    for name, path in model_paths.items()
}

# Load Sourashtra dataset
# sourashtra_df = pd.read_csv("/content/sourashtra_data.csv")
texts = sourashtra_df["Text"].astype(str).tolist()
true_labels = sourashtra_df["Label"].tolist()

# Tokenize
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

# Collect logits
with torch.no_grad():
    all_logits = []
    for model in models.values():
        outputs = model(**encodings)
        all_logits.append(outputs.logits)

# Soft voting: average logits
mean_logits = torch.mean(torch.stack(all_logits), dim=0)
preds = torch.argmax(mean_logits, dim=1).cpu().numpy()

# Print evaluation
accuracy = accuracy_score(true_labels, preds)
print(f"\nAccuracy on Sourashtra data: {accuracy * 100:.2f}%\n")
print(classification_report(true_labels, preds, digits=4))

# Combine results into DataFrame
results_df = pd.DataFrame({
    "Text": texts,
    "Ground Truth": true_labels,
    "Predicted": preds
})

# Show top rows
print("\nSample Predictions:")
print(results_df)

# Optional: Save to CSV
# results_df.to_csv("/content/sourashtra_predictions_ensemble.csv", index=False)
results_df.to_csv("/content/drive/My Drive/5541_Results/dravidian_sourashtra_predictions_ensemble.csv", index=False)


Accuracy on Sourashtra data: 50.80%

              precision    recall  f1-score   support

           0     0.5135    0.3040    0.3819       125
           1     0.5057    0.7120    0.5914       125

    accuracy                         0.5080       250
   macro avg     0.5096    0.5080    0.4866       250
weighted avg     0.5096    0.5080    0.4866       250


Sample Predictions:
                                           Text  Ground Truth  Predicted
0                              Senian mathri se             1          1
1    Askitheno chokkat monu sethe meni me hovdu             0          0
2                       Mora sangaadi kobeem ra             0          1
3                      Togo kai pajje? Me devus             0          1
4           Enno kobeem athudu meni bathi hovde             0          1
..                                          ...           ...        ...
245                 Tu elle kerithe mogo oppaes             0          0
246                 Mogo bhara

# Fully Finetuning with Sourashtra

In [ ]:
!pip install --upgrade transformers

In [ ]:
import transformers
print(transformers.__version__)

4.51.3


In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split

sourashtra_df2= sourashtra_df.copy()
print(sourashtra_df2["Label"].value_counts())

# ✅ Train-test split
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(
    sourashtra_df2["Text"], sourashtra_df2["Label"], test_size=0.2, random_state=42#, stratify=sourashtra_df2["Label"]
)

# ✅ Further split the 80% into 80% training and 20% validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_val_texts, train_val_labels, test_size=0.2, random_state=42#, stratify=train_val_labels
)
print("Train Label Distribution:")
print(train_labels.value_counts())
print("Validation Label Distribution:")
print(val_labels.value_counts())
print("Test Label Distribution:")
print(test_labels.value_counts())

# ✅ Convert to strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# ✅ Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# ✅ Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"Text": train_texts.tolist(), "Label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"Text": val_texts.tolist(), "Label": val_labels.tolist()})

# ✅ Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Remove unnecessary columns and set format to PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["Text"]).rename_column("Label", "labels")
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")

# ✅ Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Define optimizer and learning rate scheduler
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    save_total_limit=1,  # Only keep best model
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",  # 🚨 Disables wandb/logging tools
)

# ✅ Compute evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# ✅ Train model (Results will print in Colab)
trainer.train()

# ✅ Evaluate final model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)


Label
1    125
0    125
Name: count, dtype: int64
Train Label Distribution:
Label
0    85
1    75
Name: count, dtype: int64
Validation Label Distribution:
Label
1    23
0    17
Name: count, dtype: int64
Test Label Distribution:
Label
1    27
0    23
Name: count, dtype: int64


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-14-3c1a4dd10deb>:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Final Evaluation Results: {'eval_loss': 0.6236625909805298, 'eval_accuracy': 0.8, 'eval_precision': 0.7586206896551724, 'eval_recall': 0.9565217391304348, 'eval_f1': 0.8461538461538461, 'eval_runtime': 1.1357, 'eval_samples_per_second': 35.22, 'eval_steps_per_second': 2.642, 'epoch': 10.0}


In [ ]:
# Save the fine-tuned model
trainer.save_model("/content/drive/My Drive/xlm-roberta-sourashtra-best")

# Move model to Google Drive
#shutil.move("./xlm-roberta-sourashtra-best", "/content/xlm-roberta-sourashtra-best")

# Load the trained model for inference
fine_tuned_model_path = "/content/drive/My Drive/xlm-roberta-sourashtra-best"
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)

# Convert df to Hugging Face Dataset
test_dataset = Dataset.from_dict({"Text": test_texts.tolist(), "Label": test_labels.tolist()})
#dataset = Dataset.from_pandas(sourashtra_df)

# Apply tokenization
tokenized_dataset = test_dataset.map(tokenize_function, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_dataset["input_ids"])
attention_mask = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["Label"])

# Create DataLoader
batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


# Set model to evaluation mode
model.eval()

# Run inference
preds, true_labels = [], []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", zero_division=1)

# Print results
print(f"📊 Sourashtra Dataset Evaluation Results Finetuned on Sourashtra:")
print(f"➡️ Accuracy: {accuracy:.4f}")
print(f"➡️ Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")



# Combine results into DataFrame
results_df = pd.DataFrame({
    "Text": test_texts[:len(true_labels)],
    "Ground Truth": true_labels,
    "Predicted": preds
})

# Show top rows
print("\nSample Predictions:")
print(results_df)

# Optional: Save to CSV
results_df.to_csv("/content/drive/My Drive/sourashtra_finetuned_predictions.csv", index=False)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

📊 Sourashtra Dataset Evaluation Results Finetuned on Sourashtra:
➡️ Accuracy: 0.6400
➡️ Precision: 0.6452, Recall: 0.7407, F1-score: 0.6897

Sample Predictions:
                                                  Text  Ground Truth  \
142                        Thenu mogo uvo melli oppana             1   
6    Bhaat maviyor theno melli thora somaar chokkat...             1   
97                       Thora holo mogo nombike sethe             0   
60                           Thora cheledum chokkat se             1   
112                       Enno sunno madiri uskile rai             1   
181                            Tu gavaryo mogo opaaras             0   
197                    Elle bedki seeyadi mogo dhaakae             1   
184                    Haasethi tu singar ken dekaaras             0   
9                              Enno kobeem haasile rai             0   
104                           Thele betko chokkat meni             0   
200                   Mogo jhukku keshtum ken l

## Qualitative Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

baseline_df = pd.read_csv("/content/drive/My Drive/5541_Results/updated/sourashtra_predictions_baselines.csv")
finetuned_sourashtra_df = pd.read_csv("/content/drive/My Drive/5541_Results/updated/sourashtra_finetuned_predictions.csv")
ensemble_df = pd.read_csv("/content/drive/My Drive/5541_Results/updated/sourashtra_predictions_ensemble.csv")
ensemble_fewshot_df = pd.read_csv("/content/drive/My Drive/5541_Results/updated/sourashtra_predictions_fewshot_only10_ensemble.csv")

In [ ]:
ensemble_fewshot_df

Text  Ground Truth  Predicted
0                              Senian mathri se             1          1
1    Askitheno chokkat monu sethe meni me hovdu             0          0
2                       Mora sangaadi kobeem ra             0          1
3                      Togo kai pajje? Me devus             0          1
4           Enno kobeem athudu meni bathi hovde             0          1
..                                          ...           ...        ...
235                 Tu elle kerithe mogo oppaes             0          0
236                 Mogo bharaat kalathe oppaey             0          0
237              Elle nanna suno singaar ken se             0          1
238           Thone dhoo, pisi madiri dekhaaras             1          1
239               Enno kobeem askiyo hedithothe             0          1

[240 rows x 3 columns]

In [ ]:
import pandas as pd
from functools import reduce

# Clean and rename each predicted column
baseline_df = baseline_df.rename(columns={'Predicted': 'Predicted_Baseline'})
finetuned_sourashtra_df = finetuned_sourashtra_df.rename(columns={'Predicted': 'Predicted_Finetuned'})
ensemble_df = ensemble_df.rename(columns={'Predicted': 'Predicted_Ensemble'})
ensemble_fewshot_df = ensemble_fewshot_df.rename(columns={'Predicted': 'Predicted_EnsembleFewshot'})

# Normalize 'Text' and 'Ground Truth' (optional but helpful)
for df in [baseline_df, finetuned_sourashtra_df, ensemble_df, ensemble_fewshot_df]:
    df['Text'] = df['Text'].astype(str).str.strip()
    df['Ground Truth'] = df['Ground Truth'].astype(str).str.strip()

# Keep only relevant columns
dfs = [
    baseline_df[['Text', 'Ground Truth', 'Predicted_Baseline']],
    finetuned_sourashtra_df[['Text', 'Ground Truth', 'Predicted_Finetuned']],
    ensemble_df[['Text', 'Ground Truth', 'Predicted_Ensemble']],
    ensemble_fewshot_df[['Text', 'Ground Truth', 'Predicted_EnsembleFewshot']]
]

# Merge all using outer join to retain all examples
merged_df = reduce(lambda left, right: pd.merge(left, right, on=['Text', 'Ground Truth'], how='outer'), dfs)

cols_to_clean = ['Predicted_Baseline', 'Predicted_Finetuned', 'Predicted_Ensemble', 'Predicted_EnsembleFewshot']

for col in cols_to_clean:
    merged_df[col] = merged_df[col].astype(pd.Int64Dtype())

# Show result
print(merged_df)


                                         Text Ground Truth  \
0                      Aangu chokkat tholiryo            0   
1        Akaas mobbu ken se,. Mogo opparyanee            1   
2          Amba raag ken reeyadi mogo dhaakae            1   
3     Angidi kobeem menkan raan. Mogo oppana!            1   
4                 Angu bhesi madiri tholirees            1   
..                                        ...          ...   
245                      Vasuno chod avaranee            1   
246             Veti vaatho kennatha aikkuwai            1   
247     raan behys madiri chuttufiley rai ya?            1   
248                  thora chokkay chokkat se            0   
249  vambudimbu neenatha chokkat bedki ken se            0   

     Predicted_Baseline  Predicted_Finetuned  Predicted_Ensemble  \
0                     1                 <NA>                   0   
1                     1                 <NA>                   0   
2                     1                 <NA>       

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Convert Ground Truth to integers if they are strings
merged_df['Ground Truth'] = merged_df['Ground Truth'].astype(int)

for col in merged_df.columns:
    if col.startswith('Predicted_'):
        # Convert predictions to integers
        merged_df[col] = merged_df[col].astype(float)  # Ensures NaNs are still present
        merged_df_clean = merged_df.dropna(subset=['Ground Truth', col])  # Drop rows where either is NaN

        acc = accuracy_score(merged_df_clean['Ground Truth'], merged_df_clean[col])
        f1 = f1_score(merged_df_clean['Ground Truth'], merged_df_clean[col])

        print(f'{col}: Accuracy = {acc:.4f}, F1 = {f1:.2f}')

Predicted_Baseline: Accuracy = 0.5000, F1 = 0.67
Predicted_Finetuned: Accuracy = 0.6400, F1 = 0.69
Predicted_Ensemble: Accuracy = 0.5320, F1 = 0.65
Predicted_EnsembleFewshot: Accuracy = 0.6708, F1 = 0.74


## comparing baseline and finetuned

In the examples that perform better for ensemble (correctly predicted as positive), often contain words that are the same in marathi, hindi, and gujarati. For example, "hasane" in marathi and "hansana" in hindi means "laugh" whereas "haasate" in sourashtra means "laugh" as well. Sourashra sentences that contain "chokkat" are also more easily recognized as positive by the ensemble. "Cangale" in marathi is meant to be "good" and "cangale disate" is meant to be "looks good" similar in sourashtra to "chokkat dekaaras". "Dekhana" in hindi means look.

In examples the perform better for baseline (incorrectly predicted as negative), some of these sentences contain "chokkat" or "chod" which are meant to be synonyms of good, however they are also followed  by "nee" which means "not". The ensemble method seems to ignore the "nee" and incorrectly classifies as positive sentiment.

In [ ]:
# Step 1: Create correctness flags
working_df = merged_df.copy()
working_df['Baseline_Correct'] = working_df['Predicted_Baseline'] == working_df['Ground Truth']
working_df['Ensemble_Correct'] = working_df['Predicted_Ensemble'] == working_df['Ground Truth']

# Step 2: Filter rows correctly predicted by each model
correct_baseline = working_df[working_df['Baseline_Correct']]
correct_ensemble = working_df[working_df['Ensemble_Correct']]

# Step 3: Find rows correctly predicted by ensemble but not by baseline
# Compare using just the 'Text' column (or both 'Text' and 'Ground Truth' for stricter match)
ensemble_not_in_baseline = correct_ensemble[~correct_ensemble['Text'].isin(correct_baseline['Text'])]

# Show result
print("Texts correctly predicted by Ensemble but not by Baseline:")
print(ensemble_not_in_baseline[['Text', 'Ground Truth', 'Predicted_Baseline', 'Predicted_Ensemble']])

Texts correctly predicted by Ensemble but not by Baseline:
                                                 Text  Ground Truth  \
0                              Aangu chokkat tholiryo             0   
13            Bei, ella salvo thungo chokkat dekaaras             0   
25                       Eggo konni najje palakam nee             0   
27            Eggo thone kobeem photo um chokkat pode             0   
44              Elle chudidhar um tu chokkat dekaaras             0   
48                     Elle ghammu chutufiruthu oppai             0   
51                           Elle kaathayo chokkat se             0   
63                             Elle pollan chokkat se             0   
67                   Elle sawlo togo chokkat dekaaras             0   
69                 Enna hasatha vel askitheno haasate             0   
75         Enno chokkat kai thegichi aangu chokkat se             0   
83                Enno monnum kobeem chokkat yo hovde             0   
90                

In [ ]:
# First, reuse your working_df or create a new copy
working_df = merged_df.copy()

# Recompute correctness columns in case it's not already there
working_df['Baseline_Correct'] = working_df['Predicted_Baseline'] == working_df['Ground Truth']
working_df['Ensemble_Correct'] = working_df['Predicted_Ensemble'] == working_df['Ground Truth']

# Get baseline correct
correct_baseline = working_df[working_df['Baseline_Correct']]

# Get ensemble incorrect
incorrect_ensemble = working_df[~working_df['Ensemble_Correct']]

# Now get the overlap: correct in baseline but not in ensemble
baseline_not_in_ensemble = correct_baseline[correct_baseline['Text'].isin(incorrect_ensemble['Text'])]

# View results
print("Texts correctly predicted by Baseline but NOT by Ensemble:")
print(baseline_not_in_ensemble[['Text', 'Ground Truth', 'Predicted_Baseline', 'Predicted_Ensemble']])

Texts correctly predicted by Baseline but NOT by Ensemble:
                                                  Text  Ground Truth  \
1                 Akaas mobbu ken se,. Mogo opparyanee             1   
2                   Amba raag ken reeyadi mogo dhaakae             1   
14   Bhaat maviyor theno melli thora somaar chokkat...             1   
68            Elle tu meneyo, ondamaari lavaaranee ya?             1   
70                            Enne kobeem sinatey mogo             1   
77                           Enno elle jenmam urupudna             1   
107     Kondhi esani dhovro avan ya? Chokkae machi ja!             1   
111                             Me chod neenathe bedki             1   
140  Onde naano pilla melli thora somur chokkat com...             1   
141                         Onde porup melli nee thogo             1   
162                  Thego seeyadi mogo kadupu avaaras             1   
180                     Thone um dhovlo tholrithuh ya?             1   
231  

## comparing ensemble with ensemble few shot

ensemble_fewshot better:
"Umbad baathe ruchi ken se" and "Umbad baathe ruchi nee" are classified as 0 and 1 correctly. This is flipped in just the ensemble model. It is interesting because the fewshot samples contained a sentece with "nee" that is meant to be positive. Thus, the model is not just associating words with positive/negative sentiment. It is actually able to understand the context of the sentence before assigning sentiment to an extent. Similarly, "me chod neenathe bedki" is correctly classifed as negative as well but contains "chod" and "nee" which should be reflected as positive and negative respectively.


Able to identify negative sentiment much more than before

In [ ]:
# Create a fresh working copy
working_df = merged_df.copy()

# Ensure correctness columns
working_df['Ensemble_Correct'] = working_df['Predicted_Ensemble'] == working_df['Ground Truth']
working_df['Fewshot_Correct'] = working_df['Predicted_EnsembleFewshot'] == working_df['Ground Truth']

# Correct in few-shot
correct_fewshot = working_df[working_df['Fewshot_Correct']]

# Incorrect in regular ensemble
incorrect_ensemble = working_df[~working_df['Ensemble_Correct']]

# Find overlap: correct in few-shot, incorrect in ensemble
fewshot_better = correct_fewshot[correct_fewshot['Text'].isin(incorrect_ensemble['Text'])]

# View results
print("Texts correctly predicted by Few-Shot Ensemble but NOT by Regular Ensemble:")
print(fewshot_better[['Text', 'Ground Truth', 'Predicted_Ensemble', 'Predicted_EnsembleFewshot']])

Texts correctly predicted by Few-Shot Ensemble but NOT by Regular Ensemble:
                                                  Text  Ground Truth  \
2                   Amba raag ken reeyadi mogo dhaakae             1   
6                                  Aski chokkat chaley             0   
7           Askitheno chokkat monu sethe meni me hovdu             0   
16                  Cendham chokkat hujal ken dekaaras             0   
17                 Chokkat kalagasur abbe unno kaas um             0   
24                 Eggo chokkat budhi se meni kalaaras             0   
33                            Elle bedki chokkat chode             0   
42                    Elle bedko raag chokkat anchulle             0   
45           Elle dongar unju jeeyathi chokkat dekkaey             0   
46                              Elle geedhe mogo oppai             0   
59                 Elle pani peeyedi angu chokkat lave             0   
61                   Elle payasam chokkat rucci ken se      

In [ ]:
# Create a working copy again if needed
working_df = merged_df.copy()

# Make sure correctness columns exist
working_df['Ensemble_Correct'] = working_df['Predicted_Ensemble'] == working_df['Ground Truth']
working_df['Fewshot_Correct'] = working_df['Predicted_EnsembleFewshot'] == working_df['Ground Truth']

# Correct in regular ensemble
correct_ensemble = working_df[working_df['Ensemble_Correct']]

# Incorrect in few-shot
incorrect_fewshot = working_df[~working_df['Fewshot_Correct']]

# Find overlap: correct in ensemble, incorrect in few-shot
ensemble_better = correct_ensemble[correct_ensemble['Text'].isin(incorrect_fewshot['Text'])]

# Display results
print("Texts correctly predicted by Ensemble but NOT by Few-Shot Ensemble:")
print(ensemble_better[['Text', 'Ground Truth', 'Predicted_Ensemble', 'Predicted_EnsembleFewshot']])

Texts correctly predicted by Ensemble but NOT by Few-Shot Ensemble:
                                                 Text  Ground Truth  \
48                     Elle ghammu chutufiruthu oppai             0   
49                        Elle ghommo mogo oppaaranee             1   
69                 Enna hasatha vel askitheno haasate             0   
89            Esani kaayathee pod bhono mathiri hoyey             1   
90                         Gher onde dhoosi melli nee             0   
91                    Haasethi tu singar ken dekaaras             0   
127                  Mogo jhukku keshtum ken lavaaras             1   
135  Moro dhollam tu kobeem onde chokkat meni ken rai             0   
175                 Thone hudithee chodo vaatho avude             1   
182                         Thora cheledum chokkat se             1   
207                          Togo kai pajje? Me devus             0   
218            Tu haasatevel mogo sondos ken laavaras             0   
222      

In [ ]:
#below is the 10 sourashtra samples used to also train the ensemble model
fewshot_examples = merged_df[merged_df['Predicted_EnsembleFewshot'].isna()]
print("Few-shot examples (used for in-context training, no prediction made):")
print(fewshot_examples[['Text', 'Ground Truth']])

Few-shot examples (used for in-context training, no prediction made):
                                                  Text  Ground Truth
49                         Elle ghommo mogo oppaaranee             1
89             Esani kaayathee pod bhono mathiri hoyey             1
121                              Mogo elle geedh oppai             0
135   Moro dhollam tu kobeem onde chokkat meni ken rai             0
145                        Phul vaasano chokkat avaras             0
149  Pos podethe vel onde padam bithir saate chod l...             0
166                           Thele betko chokkat meni             0
175                  Thone hudithee chodo vaatho avude             1
222           Tu kanadee thalyedi pisi madiri dekaaras             1
229                            Tu kobeem thapu ken jai             1


## comparing ensemble few shot with full finetuned on sourashtra

the finetuned model on sourashtra only tends to classify emotions/feelings like "kadapu" and "nombike" and "porup" which are now typically adpoted from more dravidian languages especially Tamil. The ensemble fewshot model misclassifies these, but identifies other sentences correctly that have words more linguistically similar.

Negative tone detection: Many of these examples contain words that hint at negativity, criticism, or insult, such as:

bedki, waste, chod, dekhaana, ghendo — these could resemble insults or negative sentiment.

These might be common in Hindi or Marathi slang too

In [ ]:
# Filter to only rows where both predictions exist
valid_rows = merged_df.dropna(subset=['Predicted_Finetuned', 'Predicted_EnsembleFewshot']).copy()

# Now it's a safe copy, no warnings from this point
valid_rows.loc[:, 'Fewshot_Correct'] = valid_rows['Predicted_EnsembleFewshot'] == valid_rows['Ground Truth']
valid_rows.loc[:, 'Finetuned_Correct'] = valid_rows['Predicted_Finetuned'] == valid_rows['Ground Truth']

# Get samples where fewshot is right but finetuned is not
fewshot_beats_finetuned = valid_rows[
    (valid_rows['Fewshot_Correct']) & (~valid_rows['Finetuned_Correct'])
]

# Display results
fewshot_beats_finetuned[['Text', 'Ground Truth', 'Predicted_Finetuned', 'Predicted_EnsembleFewshot']]

Text  Ground Truth  \
39                    Elle bedki ven podjaal pode             1   
40    Elle bedko kobeem chod neenatha vaatha kere             1   
108  Kuppo angu thayedi melli thogo chod dekhaana             1   
225            Tu kobeem ghendo waste kele rai ya             1   
236                                Tu onde ghedat             1   
249      vambudimbu neenatha chokkat bedki ken se             0   

     Predicted_Finetuned  Predicted_EnsembleFewshot  
39                   0.0                        1.0  
40                   0.0                        1.0  
108                  0.0                        1.0  
225                  0.0                        1.0  
236                  0.0                        1.0  
249                  1.0                        0.0

In [ ]:
# Filter rows where both predictions are present (you've probably already done this)
valid_rows = merged_df.dropna(subset=['Predicted_Finetuned', 'Predicted_EnsembleFewshot']).copy()

# Recalculate correctness with .loc to avoid warnings
valid_rows.loc[:, 'Fewshot_Correct'] = valid_rows['Predicted_EnsembleFewshot'] == valid_rows['Ground Truth']
valid_rows.loc[:, 'Finetuned_Correct'] = valid_rows['Predicted_Finetuned'] == valid_rows['Ground Truth']

# Now filter for Finetuned correct & Fewshot incorrect
finetuned_beats_fewshot = valid_rows[
    (valid_rows['Finetuned_Correct']) & (~valid_rows['Fewshot_Correct'])
]

# Display the relevant info
finetuned_beats_fewshot[['Text', 'Ground Truth', 'Predicted_Finetuned', 'Predicted_EnsembleFewshot']]

Text  Ground Truth  \
14   Bhaat maviyor theno melli thora somaar chokkat...             1   
30                    Elle angidi um katheyo chod abbe             0   
82                             Enno kobeem haasile rai             0   
85                                   Enno porup ken se             0   
162                  Thego seeyadi mogo kadupu avaaras             1   
188                      Thora holo mogo nombike sethe             0   
217                            Tu gavaryo mogo opaaras             0   

     Predicted_Finetuned  Predicted_EnsembleFewshot  
14                   1.0                        0.0  
30                   0.0                        1.0  
82                   0.0                        1.0  
85                   0.0                        1.0  
162                  1.0                        0.0  
188                  0.0                        1.0  
217                  0.0                        1.0

**Finetuned Model**

Accuracy: 62%

Balanced: Performs decently on both classes.

Recall (0): 50% → catches half of the negatives.

Recall (1): 72% → strong at identifying positives.

F1 Score (0 vs. 1): 0.55 vs. 0.67 → stronger on positive class, but decent on both.

**Few-shot Ensemble**

Accuracy: 60%

Recall (1): Very strong at 84% → rarely misses positives.

Recall (0): Weak at 32% → misses most of the negatives.

F1 Score (0 vs. 1): 0.42 vs. 0.69 → strong on positive, much weaker on negative.


Where the finetuned model underperforms (on positive class), the few-shot model shines.

And where the few-shot model struggles (on negative class), the finetuned model does better.

The few-shot ensemble may have learned strong positive cues from related languages. The finetuned model, being trained only on limited Sourashtra, might generalize less aggressively but be more cautious (hence better at negatives).

This complementarity could be a great justification for a smarter ensemble or meta-classifier that weighs predictions differently per class or context.

In [ ]:
valid_rows = merged_df.dropna(subset=['Predicted_Finetuned', 'Predicted_EnsembleFewshot']) #gets rid of at most 210 samples


from sklearn.metrics import confusion_matrix, classification_report

# Ground truth
y_true = valid_rows['Ground Truth'].astype(int)

# Finetuned predictions
y_pred_finetuned = valid_rows['Predicted_Finetuned'].astype(int)

# Few-shot Ensemble predictions
y_pred_fewshot = valid_rows['Predicted_EnsembleFewshot'].astype(int)

print("Finetuned Model:")
print(confusion_matrix(y_true, y_pred_finetuned))
print(classification_report(y_true, y_pred_finetuned))

print("Few-shot Ensemble:")
print(confusion_matrix(y_true, y_pred_fewshot))
print(classification_report(y_true, y_pred_fewshot))

Finetuned Model:
[[11 11]
 [ 7 18]]
              precision    recall  f1-score   support

           0       0.61      0.50      0.55        22
           1       0.62      0.72      0.67        25

    accuracy                           0.62        47
   macro avg       0.62      0.61      0.61        47
weighted avg       0.62      0.62      0.61        47

Few-shot Ensemble:
[[ 7 15]
 [ 4 21]]
              precision    recall  f1-score   support

           0       0.64      0.32      0.42        22
           1       0.58      0.84      0.69        25

    accuracy                           0.60        47
   macro avg       0.61      0.58      0.56        47
weighted avg       0.61      0.60      0.56        47

